I'd like to save datastamp of previous vesion of lab2, so I have not delete them. 

In [9]:
import pandas as pd
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils
url="lab2_final_ver.csv"
dataset = pd.read_csv(url, sep='|', encoding='UTF-8')
dataset_len = len(dataset)

###############################################################################################################################
According the structure of dataset
Because of the trouble with finding a context of hyponyms in some cases (which was discussed on previous Wed, I added one more row 'hyponym2'. If it was possible to find a direct context hyponym (from the task) and hyperonym, the values of rows are the same. But in case of inability to get this pair it was replaced by hyperonim. (Smthg like Plenty in Plenty)
##############################################################################################################################

In [10]:
print(dataset.head(5))

         hyponim       hyponim1                       hyperonim  \
0  Капельмейстер  Капельмейстер    руководитель хоровой капеллы   
1  Капельмейстер  Капельмейстер                         дирижер   
2  Капельмейстер  Капельмейстер  руководитель военного оркестра   
3  Капельмейстер        дирижер                      профессия    
4  Капельмейстер      Профессия      род трудовой деятельности    

                                                text  \
0  Капельмейстер - это руководитель хоровой капел...   
1  Капельмейстер - дирижер военного духового орке...   
2     Капельмейстер - руководитель военного оркестра   
3  Дирижер - профессия штучная, эксклюзивная, кот...   
4   Профе́ссия (лат. professio; от profiteor — «о...   

                                                link  bool  \
0  https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...   1.0   
1  https://kartaslov.ru/%D0%B7%D0%BD%D0%B0%D1%87%...   1.0   
2  https://kartaslov.ru/%D0%B7%D0%BD%D0%B0%D1%87%...   1.0   
3           

In [11]:
START = or_(rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN')),
        rule(gram('ADVB'))
    ).optional(),
    gram('NOUN'),
    or_(
        rule(gram('ADJF')).repeatable(),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')  
    ),
    rule(gram('NOUN')))

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),
    or_(
        eq('принадлежит')
    )
)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также'),
    eq('весьма')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    ),
    rule(
        eq('-')
    ),
    rule(
        eq('–'),
        eq('это'),
        eq('не')
    ),
    rule(
    eq('является')
    ),
    rule(
        eq('-'),
        eq('это')
    ),
    rule(
    or_
        (eq('называют'),
        eq('называется'),
        eq('называли'))
    ),
    rule(
    eq('считается')
    ),
    rule(eq('то'),
        eq('есть')),
    rule(
    eq(','),
    eq('как')
    ),
    rule(
    eq('один'),
    eq('из')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable(2,3)
    ), 
    rule(
        gram('ADJF'),
        gram('NOUN')
    ),
    rule(
        gram('ADVB'),
        gram('ADJF'),
        gram('NOUN')
    )
)


Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = or_(rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
        ))

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)



In [12]:
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()
def plur(word):
    try:
        b = morph.parse(word)[0]
        c = b.inflect({'plur'}) 
        return (c.word)
    except: 
        return (word)

This part of code is faster because it compare one value of hyponym with one value of text correspondingly 

In [13]:
#Эта часть кода удаляет продию в случае, если в слове не встречается 'й'
#1) токенизировать корпус
#2) проверить наличие й
#3) в случае, если й отсутствует - удалить ударение 
#4) вернуть массив в строку
def delete_pros(text):
    array = []
    for ind,w in enumerate(text.split(" ")):
        array.append(w)
    for j in range(len(array)):
        if (not(re.search(r'\й', array[j]))):
            array[j] = utils.deaccent(array[j])
    return (' '.join(array))

In [14]:
import numpy as np
def get_hyperonyms_rw1(i): 
    word = dataset['hyponim1'][i]
    array_of_main_words = []
    array_of_main_words.append(word.lower())
    if (plur(word) != word.lower()):
        array_of_main_words.append(plur(word))
    array_of_hyper = []
    for main_word in array_of_main_words:    
        #HYPONYM = eq(utils.deaccent(main_word))
        if (not(re.search(r'\й', main_word))):
            HYPONYM = eq(utils.deaccent(main_word))
        else:
            HYPONYM = eq(main_word)
        RULE = or_(
            rule(HYPONYM, ATAKJE, START, MID, END),
            rule(HYPONYM, MID, END),
            rule(START_S, END, KAK, HYPONYM),
            rule(END, INCLUDING, HYPONYM),
            rule(HYPONYM, MID, END, END),
            rule(HYPONYM, MID, START, END),
            rule(START, MID, HYPONYM),
            rule(HYPONYM, END)
            )
        parser = Parser(RULE) 
        text = dataset['text'][i]
        text = delete_pros(text)
        text = re.sub(r'\(.+?\)', '', text)
        text = text.lower().replace('* сергии радонежскии* ', '')
    
        for idx, match in enumerate(parser.findall(text.lower())):
            k = [_.value for _ in match.tokens]                
            array_of_hyper.append(k)
    if (len(array_of_hyper) > 0):
        print(text)
        print(array_of_hyper)

In [15]:
number_of_words = 0
for i in range(len(dataset)):
    get_hyperonyms_rw1(i)
    if(dataset['bool'][i] == 0):
        print('this word was not find in context as a pair "hyponym + hyperonim", there is a reason why')
        print(dataset['comment'][i])


капельмейстер - это руководитель хоровой капеллы и/или оркестра.
[['капельмейстер', '-', 'это', 'руководитель', 'хоровой', 'капеллы', 'и']]
капельмейстер - дирижер военного духового оркестра.
[['капельмейстер', '-', 'дирижер', 'военного', 'духового', 'оркестра']]
капельмейстер - руководитель военного оркестра
[['капельмейстер', '-', 'руководитель', 'военного', 'оркестра']]
 профессия  — род трудовой деятельности человека, обычно его источник существования. 
[['профессия', '—', 'род', 'трудовой', 'деятельности', 'человека']]
this word was not find in context as a pair "hyponym + hyperonim", there is a reason why
капельмейстер=>дирижер=>профессия
 каперсы - род растений семейства каперсовые , иногда род относят к семейству капустные .
[['каперсы', '-', 'род', 'растений', 'семейства']]
 каперсы — нераспустившиеся бутоны колючего кустарника capparis spinosa.
[['каперсы', '—', 'нераспустившиеся', 'бутоны', 'колючего', 'кустарника']]
каперс - забытый овощ. многие хозяйки, которым попадались 

крупное морское судно - средство передвижения, на котором путешествуют или перевозят что-либо по воде или по воздуху
[['судно', '-', 'средство', 'передвижения']]
this word was not find in context as a pair "hyponym + hyperonim", there is a reason why
Карбас =>судно=>средство передижения
карбиды — соединения металлов и неметаллов с углеродом. традиционно к карбидам относят соединения, где углерод имеет большую электроотрицательность, чем второй элемент 
[['карбиды', '—', 'соединения', 'металлов', 'и', 'неметаллов', 'с', 'углеродом']]
карбиды — тугоплавкие твердые вещества. они нелетучи и не растворимы ни в одном из известных растворителей. карбиды бора и кремния , титана, вольфрама, циркония  обладают высокой твердостью, жаростойкостью, химической инертностью.
[['карбиды', '—', 'тугоплавкие', 'твердые', 'вещества'], ['карбиды', 'бора', 'и', 'кремния']]
карбиды — тугоплавкие твердые вещества. они нелетучи и не растворимы ни в одном из известных растворителей. карбиды бора и кремния , тит

картуз — мужской головной убор, имевший распространение в xvi — начале xx века в северной и восточной европе .
[['картуз', '—', 'мужской', 'головной', 'убор']]
картуз - бумажный мешок для чего-н. сыпучего устар..
[['картуз', '-', 'бумажный', 'мешок']]
картуз - фуражка с козырьком
[['картуз', '-', 'фуражка', 'с', 'козырьком']]
продажа одежды, обуви и аксессуаров - фуражка
[['обуви', 'и', 'аксессуаров', '-', 'фуражка']]
this word was not find in context as a pair "hyponym + hyperonim", there is a reason why
картуз=>фуражка=>одежда
кастелянша - это сотрудница, ведающая хранением и выдачей одежды и белья.
[['кастелянша', '-', 'это', 'сотрудница']]
кастелянша это такая профессия, принято считать, что этот человек отвечает за одежду или белье.
[['кастелянша', 'это', 'такая', 'профессия']]
кастелянша считается женским родом деятельности
[['кастелянша', 'считается', 'женским', 'родом']]
кастелянша - сотрудник бельевой компании
[['кастелянша', '-', 'сотрудник', 'бельевой', 'компании']]
кастомиз

Удалила в ручную просодию со слова капельмейстер в датасете